In [4]:
#[1]Step 
#1.understand the shape of the data(Histograms, box plots, etc.)
#2.Data cleaning
#3.Data Exploration
#4.Feature Engineering
#5.Data Preprocessing for Model
#6.Basic Model Building
#7.Model Tuning
#8.Ensemble Model Building
#9.Results 

In [5]:
#[2]Project Planning 
#1.Understand nature of the data.info() .describe()
#2.Histograms and boxplots
#3.Value counts
#4.Missing data
#5.Correlation between the metrics
#6.Explore interesting themes
    #1)Wealthy survive?   
    #2)By location such as: 
    #3)Age scatterplot with ticket price 
    #4)Young and wealthy Variable?
    #5)Total spent? 
#7.Feature engineering
#8.preprocess data together or use a transformer? 
    #1)Use label for train and test
#10.Scaling? 
#11.Model baseline
#12.Model comparison with CV    


In [6]:
# Light data expoloration 

#1 For numeric data 
#-made Histogram to understand distributions 
#corrplot
#pivot table coparing survival rate across numeric variables

#2 For Categorical Data
#made bar charts to understand balance of classes
#made pivot tables to understand relationship with survival
#


In [7]:
#Data Dictionary 

In [8]:
#Data preprocessing for model 
#1.Drop null values from Embarked(only 2? 3?)
#Include only relevant variables (Since we have limited data,
#2.i wanted to exclude things like name and passanger ID so that we could have a resonable number of features for our models to deal with 
        #Variables : Pclass sex age sitSp parch fare embarked cabin_adv cavbin_multiple numeric_ticket name_title
#3.Do catagorical transforms on all data. usually we would use a transformer, but with this approach with can ensure the shape of the test dadta through this method. 
#4.impute data with mean for fare and age(shold also experiment with median)
#5.Normalized fare using logarithm to give more semblance of a normal distribution
#6.Scaled data 0-1 with standard scaler

Importing the Dependencise

In [39]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

import warnings 
warnings.filterwarnings('ignore')

Data Collection and Processing

In [43]:
# load the data from csv file to Pandas DataFrame 
titanic_test_df = pd.read_csv("./titanic_test.csv")
titanic_train_df = pd.read_csv("./titanic_train.csv")

In [44]:
#printing the first 5 rows of the Dataframe 
print("train data colomns :", titanic_train_df.shape ,"\n"
      , "test data colomns :", titanic_test_df.shape)
titanic_train_df.head()

train data colomns : (891, 12) 
 test data colomns : (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [45]:
#number of rows and Columns 
titanic_train_df.shape

(891, 12)

In [46]:
#getting some informations about the data 
titanic_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [47]:
#chect the number of missing values in each column 
titanic_train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [50]:
# 문자열 => 숫자형으로 인코딩 함수
def encode_feature(df=None):
    features = ["Sex","Cabin","Embarked"]
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

In [51]:
# Data frame capy and delete colomn data 
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    df_copy.drop(["Name", "Ticket"], axis=1, inplace = True) # Time 컬럼 삭제
    return df_copy

In [52]:
# 원본 데이터를 학습데이터와 테스트데이터를 분리하는 함수 선언
from sklearn.model_selection import train_test_split

def get_train_test_dataset(df=None):
    # 인코딩 함수 호출
    df_copy = encode_feature(df)
    # 입력된 df을 복사하고, Time을 삭제
    df_copy = get_preprocessed_df(df)
    # 데이터와 레이블 분리
    X_features = df_copy.drop("Survived", axis=1)  # 답 피처 제거
    y_labels = df_copy["Survived"] # 답만 추출
    X_train, X_test, y_train, y_test = train_test_split(
          X_features
        , y_labels
        , test_size= 0.2
        , random_state= 0
        , stratify= y_labels  
    )
    return X_train, X_test, y_train, y_test

In [53]:
titanic_test_df = encode_feature(titanic_test_df)
titanic_test_df = get_preprocessed_df(titanic_test_df)

In [54]:
X_train, X_test, y_train, y_test = get_train_test_dataset(titanic_train_df)

Handling and Missing

In [55]:
#Drop the "Cabin" column form the Dataframe 
titanic_data = titanic_test_df.drop(columns='Cabin', axis=1, inplace=True)

In [57]:
#Replacing the missing values in "AGE" column with mean value 
titanic_test_df['Age'].fillna(titanic_data['Age'].mean(), inplace=True)

TypeError: 'NoneType' object is not subscriptable

In [58]:
#Finding the mode value of "Embarked" column
print(titanic_data['Embarked'].mode())

TypeError: 'NoneType' object is not subscriptable

In [ ]:
print(titanic_data['Embarked'].mode()[0])

NameError: name 'titanic_data' is not defined

In [ ]:
#Check the number of missing values in each column 
titanic_data.isnull().sum()         ##왜 2가 나오냐 0 나와야하는데..

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

Data Analysis 

In [ ]:
#Getting some statistical measures about the data 
titanic_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
#finding the number of people survived and not survived 
titanic_data['Survived'].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

In [ ]:
titanic_data['Sex'].value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

Data Visualization 

In [59]:
sns.set()

In [18]:
#making a count plot for "survived" column
sns.barplot(x='Survived', data=titanic_data)

ValueError: Could not interpret value `Survived` for `x`. Value is a string, but `data` was not passed.

In [ ]:
#making a count plot for "Sex" column
sns.barplot('Sex', data=titanic_data)

TypeError: barplot() got multiple values for argument 'data'

In [ ]:
#number of survivors Gender wise 
sns.barplot('Sex', hue='Servived', data=titanic_data)

TypeError: barplot() got multiple values for argument 'data'

In [ ]:
#making a count plot for "Pclass" column
sns.countplot('Pclass',data=titanic_data)

Encoding the Categorical Columns

In [ ]:
titanic_data['Sex'].value_counts()

In [ ]:
titanic_data['Embarked'].value_counts()

In [ ]:
titanic_data.head()

In [ ]:
#Converting categorical Columns     # 왜 float 으로 나오냐
titanic_data.replace({'Sex':{'male':0,'female':1}, 'Embarked':{'S':0,'C':1,'Q':2}}, inplace=True)

In [ ]:
titanic_data.head()

In [ ]:
X= titanic_data.drop(columns= ['PassengerId','Name','Ticket','Survived'], axis=1)
Y= titanic_data['Survived']

In [ ]:
print(X)

In [ ]:
print(Y)

Splitting the data into training data and Test data 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=156)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

Model Training 

LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
#training the Logistic Regression model with training data 
model.fit(X_train,Y_train)              #??? 

Model Evalution

Accuracy

In [ ]:
#accuracy on training data 
X_train_prediction = model.predict(X_train)

In [ ]:
print(X_train_prediction)

In [ ]:
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score of training data :', training_data_accuracy)

In [ ]:
#accuracy on test data 
X_train_prediction = model.predict(X_test)

In [ ]:
test_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score of test data :', test_data_accuracy)